In [ ]:
import cv2
import numpy as np
import os 
import tensorflow as tf

In [ ]:
new_model = tf.keras.models.load_model('classifier')

cascadePath = "haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascadePath)
font = cv2.FONT_HERSHEY_SIMPLEX

id = 0
names = ["imposter", "valid user"]
# Initialize and start realtime video capture
cam = cv2.VideoCapture(0)
cv2.startWindowThread()

cam.set(3, 640) # set video widht
cam.set(4, 480) # set video height
# Define min window size to be recognized as a face
minW = 0.1*cam.get(3)
minH = 0.1*cam.get(4)
while True:
    ret, img =cam.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    faces = faceCascade.detectMultiScale( 
        gray,
        scaleFactor = 1.2,
        minNeighbors = 5,
        minSize = (int(minW), int(minH)),
       )
    for(x,y,w,h) in faces:
        cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)
        face = cv2.resize(gray[y:y+h,x:x+w], dsize=(150, 150), interpolation=cv2.INTER_CUBIC)
        face = tf.keras.utils.array_to_img(
            face, data_format=None, scale=True, dtype=None
        )
        id = new_model.predict(face)

        if (id == 1):
            id = "valid user"
        else:
            id = "imposter"
        
        cv2.putText(
                    img, 
                    str(id), 
                    (x+5,y-5), 
                    font, 
                    1, 
                    (255,255,255), 
                    2
                   )
        cv2.putText(
                    img, 
                    str(id), 
                    (x+5,y+h-5), 
                    font, 
                    1, 
                    (255,255,0), 
                    1
                   )  
    
    cv2.imshow('camera',img) 
    k = cv2.waitKey(10) & 0xff # Press 'ESC' for exiting video
    if k == 27:
        break
# Do a bit of cleanup
print("\n [INFO] Exiting Program and cleanup stuff")
cam.release()
cv2.waitKey(1)
cv2.destroyAllWindows()
cv2.waitKey(1)